In [10]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import os

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [6]:
BATCH_SIZE = 64
NUM_EPOCHS = 50
LEARNING_RATE = 0.001

In [11]:
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((48, 48)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [12]:
train_dataset = datasets.ImageFolder("/kaggle/input/fer2013/train", transform=transform)
val_dataset   = datasets.ImageFolder("/kaggle/input/fer2013/test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Residual Block
torch.manual_seed(42)
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, dropout_prob=0.3):
        super().__init__()
        self.shortcut = nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm2d(out_channels)
        self.pool  = nn.MaxPool2d(2,2)
        self.drop  = nn.Dropout2d(dropout_prob)

    def forward(self, x):
        s = self.shortcut(x)
        y = F.relu(self.bn1(self.conv1(x)))
        y = self.bn2(self.conv2(y))
        out = F.relu(s + y)
        out = self.pool(out)
        return self.drop(out)

In [13]:
class EmotionCNN(nn.Module):
    def __init__(self, num_classes=7):
        super().__init__()
        self.rb1 = ResidualBlock(1,  32)
        self.rb2 = ResidualBlock(32, 64)
        self.rb3 = ResidualBlock(64, 128)
        self.rb4 = ResidualBlock(128,256)
        self.global_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc1 = nn.Linear(256, 128)
        self.bn_fc = nn.BatchNorm1d(128)
        self.drop_fc = nn.Dropout(0.5)
        self.out = nn.Linear(128, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.rb1(x)
        x = self.rb2(x)
        x = self.rb3(x)
        x = self.rb4(x)
        x = self.global_pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.bn_fc(self.fc1(x)))
        x = self.drop_fc(x)
        return self.out(x)

In [14]:
model = EmotionCNN(num_classes=7).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


In [15]:
for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_acc = 100. * correct / total
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}]  Loss: {avg_loss:.4f}  Accuracy: {train_acc:.2f}%")

# Save model
torch.save(model.state_dict(), "emotion_cnn_resnet.pt")
print("\nModel saved as emotion_cnn_resnet.pt")


Epoch [1/50]  Loss: 2.0431  Accuracy: 20.09%
Epoch [2/50]  Loss: 1.8384  Accuracy: 23.32%
Epoch [3/50]  Loss: 1.8004  Accuracy: 25.08%
Epoch [4/50]  Loss: 1.7882  Accuracy: 25.73%
Epoch [5/50]  Loss: 1.7651  Accuracy: 27.11%
Epoch [6/50]  Loss: 1.6961  Accuracy: 30.59%
Epoch [7/50]  Loss: 1.5487  Accuracy: 39.17%
Epoch [8/50]  Loss: 1.4590  Accuracy: 43.69%
Epoch [9/50]  Loss: 1.3946  Accuracy: 46.31%
Epoch [10/50]  Loss: 1.3460  Accuracy: 48.63%
Epoch [11/50]  Loss: 1.3099  Accuracy: 49.95%
Epoch [12/50]  Loss: 1.2806  Accuracy: 51.33%
Epoch [13/50]  Loss: 1.2577  Accuracy: 51.69%
Epoch [14/50]  Loss: 1.2310  Accuracy: 53.20%
Epoch [15/50]  Loss: 1.2218  Accuracy: 53.59%
Epoch [16/50]  Loss: 1.1966  Accuracy: 54.84%
Epoch [17/50]  Loss: 1.1838  Accuracy: 54.76%
Epoch [18/50]  Loss: 1.1678  Accuracy: 55.59%
Epoch [19/50]  Loss: 1.1607  Accuracy: 56.13%
Epoch [20/50]  Loss: 1.1502  Accuracy: 56.30%
Epoch [21/50]  Loss: 1.1399  Accuracy: 57.09%
Epoch [22/50]  Loss: 1.1266  Accuracy: 57.6